## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import ReduceLROnPlateau
#from keras.backend.tensorflow_backend import set_session



Using TensorFlow backend.


In [2]:
# CUDNN_STATUS_INTERNAL_ERROR 於運行時出現, 重裝 cudnn 之後無效, suspect gpu ram issue
# 利用動態分配GPU內存 problem solved.
# ref: https://github.com/keras-team/keras/issues/4161
def dynam_GPU_ram():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
    config.log_device_placement = True  # to log device placement (on which device the operation ran)
    sess = tf.compat.v1.Session(config=config)
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

    
    
dynam_GPU_ram()

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5



In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
    mean = np.mean(X_train,axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7)
    return X_train, X_test, mean, std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [6]:
input_shape = x_train.shape[1:]

# Define keras callback function
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.4, 
                                            min_lr=1e-8)
callbacks_list = [learning_rate_reduction]

#Start
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), strides=1, input_shape=input_shape, padding='same', activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(256)) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=256,epochs=20,validation_split=0.15, callbacks=callbacks_list)

/media/ychuang/Data/ych/venv_list/dl_marathon/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Train on 42500 samples, validate on 7500 samples
Epoch 1/20
42500/42500 [==============================] - 6s 148us/step - loss: 5.9817 - accuracy: 0.4393 - val_loss: 4.0395 - val_accuracy: 0.2573
Epoch 2/20
42500/42500 [==============================] - 6s 144us/step - loss: 2.1862 - accuracy: 0.5638 - val_loss: 1.3471 - val_accuracy: 0.5353
Epoch 3/20
42500/42500 [==============================] - 6s 142us/step - loss: 1.0148 - accuracy: 0.6615 - val_loss: 1.4738 - val_accuracy: 0.5449
Epoch 4/20
42500/42500 [==============================] - 6s 142us/step - loss: 1.0089 - accuracy: 0.6727 - val_loss: 1.1223 - val_accuracy: 0.6259
Epoch 5/20
42500/42500 [==============================] - 6s 141us/step - loss: 0.8303 - accuracy: 0.7118 - val_loss: 1.0687 - val_accuracy: 0.6503
Epoch 6/20
42500/42500 [==============================] - 6s 143us/step - loss: 0.7919 - accuracy: 0.7261 - val_loss: 1.1409 - val_accuracy: 0.6229
Epoch 7/20
42500/42500 [==============================] - 6s 14

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
y_pred = classifier.predict(x_test).argmax(-1)

In [8]:
np.sum(y_test.argmax(-1) == y_pred)/y_test.shape[0]

0.6596